In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from pycaret.regression import RegressionExperiment

from etl import ETL
from feature_creation import FeatureCreation

plt.style.use("seaborn-v0_8")

In [2]:
df_yield = pd.read_csv("data/barley_yield_from_1982.csv", sep=";")  # 1982 to 2018
df_climate = pd.read_parquet(
    "data/climate_data_from_1982.parquet"
)  # 1982-2014 2015-2050

In [3]:
df_yield, df_climate = ETL(df_yield, df_climate).run()

--- df_climate---
Departments/Scenario dropped because of any missing values:                                    nom_dep  scenario
date                                                
2015-01-01 12:00:00               Calvados  ssp2_4_5
2015-01-01 12:00:00            Deux_Sevres  ssp2_4_5
2015-01-01 12:00:00                Essonne  ssp2_4_5
2015-01-01 12:00:00                   Eure  ssp2_4_5
2015-01-01 12:00:00                  Rhone  ssp2_4_5
2015-01-01 12:00:00        Tarn_et_Garonne  ssp2_4_5
2015-01-01 12:00:00  Territoire_de_Belfort  ssp2_4_5
2015-01-01 12:00:00               Vaucluse  ssp2_4_5
--- df_yield ---
Departments dropped because of almost absolute absence of data:
 ['Hauts_de_Seine' 'Paris' 'Seine_SeineOise']


In [4]:
df_hist, df_forecast = FeatureCreation(df_yield, df_climate).run()

--- Amplitude feature created ---


# Prepare data


In [ ]:
df = df_hist[
    ~df_hist.isna().any(axis=1)
]  # Remove rows where we have yield but no climate data
target = "yield"
# df = df.drop(columns=["department"])

In [7]:
df

,year,yield,area,production,amp_daily_NSA_temp_Apr,amp_daily_NSA_temp_Aug,amp_daily_NSA_temp_Dec,amp_daily_NSA_temp_Feb,amp_daily_NSA_temp_Jan,amp_daily_NSA_temp_Jul,...,amp_precipitation_Dec,amp_precipitation_Feb,amp_precipitation_Jan,amp_precipitation_Jul,amp_precipitation_Jun,amp_precipitation_Mar,amp_precipitation_May,amp_precipitation_Nov,amp_precipitation_Oct,amp_precipitation_Sep
0,1982,3.950080,16065.0,63458.0,14.585327,13.266602,14.508118,12.605316,8.102631,8.545471,...,0.000337,0.000305,0.000293,0.000355,0.000220,0.000566,0.000520,0.000488,0.000870,0.000160
1,1983,2.648276,14500.0,38400.0,11.894348,7.150391,16.380951,17.021851,9.464539,10.034546,...,0.000416,0.000397,0.000439,0.000694,0.000392,0.000536,0.000355,0.000363,0.000217,0.000531
2,1984,4.822580,15500.0,74750.0,14.128510,5.867737,12.289917,9.180328,15.847443,10.030670,...,0.000269,0.000420,0.000419,0.000635,0.000335,0.000334,0.000682,0.000384,0.000519,0.000394
3,1985,4.196770,15500.0,65050.0,11.441620,8.075775,14.267517,13.399414,14.638794,6.790100,...,0.000385,0.000473,0.000610,0.000887,0.000538,0.000358,0.000422,0.000802,0.000813,0.000430
4,1986,3.598450,12900.0,46420.0,11.271729,10.372467,9.982452,11.505005,10.697601,9.380066,...,0.000542,0.000221,0.000214,0.000724,0.000288,0.000266,0.000588,0.000690,0.000190,0.000420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3469,2010,7.037840,9624.0,67732.2,9.208282,10.101349,10.645935,9.205994,9.430847,11.243744,...,0.000132,0.000088,0.000164,0.000360,0.000029,0.000333,0.000124,0.000203,0.000100,0.000164
3470,2011,6.155870,8766.0,53962.4,9.366638,13.055542,8.522217,10.069000,13.514313,11.270905,...,0.000164,0.000097,0.000088,0.000159,0.000481,0.000235,0.000303,0.000152,0.000306,0.000166
3471,2012,7.675490,9100.0,69847.0,11.627319,11.498718,12.754639,10.042389,16.795593,10.162323,...,0.000125,0.000105,0.000137,0.000035,0.000042,0.000134,0.000232,0.000212,0.000238,0.000295
3472,2013,7.043820,10360.0,72974.0,6.623810,11.658661,7.555054,10.872925,11.944244,13.028351,...,0.000227,0.000186,0.000106,0.000235,0.000178,0.000162,0.000146,0.000175,0.000225,0.000046


# Baseline model comparison


We will use pycaret to quickly compare the performance of different models on the dataset before actually selecting the best model for further tuning.


In [ ]:
s = RegressionExperiment()
s.setup(df, target=target, session_id=123)

In [12]:
best = s.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.2227,0.1076,0.3275,0.9424,0.0617,0.0500,0.0090
lightgbm,Light Gradient Boosting Machine,0.3138,0.1660,0.4065,0.9113,0.0806,0.0732,0.1840
et,Extra Trees Regressor,0.3456,0.2034,0.4502,0.8917,0.0885,0.0806,0.1440
rf,Random Forest Regressor,0.3594,0.2161,0.4638,0.8848,0.0911,0.0840,0.4050
gbr,Gradient Boosting Regressor,0.3746,0.2324,0.4809,0.8761,0.0942,0.0872,0.3050
lr,Linear Regression,0.4428,0.3197,0.5638,0.8298,0.1099,0.1033,0.2360
ridge,Ridge Regression,0.4448,0.3229,0.5670,0.8280,0.1106,0.1040,0.0080
br,Bayesian Ridge,0.4446,0.3229,0.5670,0.8280,0.1107,0.1040,0.0080
ada,AdaBoost Regressor,0.4784,0.3544,0.5944,0.8108,0.1099,0.1061,0.0940
lar,Least Angle Regression,0.4719,0.3625,0.6013,0.8066,0.1155,0.1089,0.0070


/opt/homebrew/Caskroom/miniconda/base/envs/automl/lib/python3.9/site-packages/pycaret/internal/pycaret_experiment/supervised_experiment.py:323: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  master_display_.apply(


# Train Test Split


In [25]:
target = "yield"
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [30]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import get_scorer, get_scorer_names

In [ ]:
knn = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("knn", KNeighborsRegressor(n_neighbors=5)),
    ]
)
knn.fit(X_train, y_train)

score_names = [
    "r2",
    "neg_mean_absolute_error",
    "neg_root_mean_squared_error",
    "neg_mean_absolute_percentage_error",
]
for name in score_names:
    scorer = get_scorer(name)
    print(name, " : ", scorer(knn, X_test, y_test))

r2  :  0.7848827021327762
neg_mean_absolute_error  :  -0.5076098440867333
neg_root_mean_squared_error  :  -0.6758594197637094
neg_mean_absolute_percentage_error  :  -0.12951819688390978


# Test using df_forecast
